# <center> Kafka Producer for Twitter </center>

**Acquire and decompress Kafka**

```
$ wget http://download.nextag.com/apache/kafka/1.0.0/kafka_2.11-1.0.0.tgz
$ tar xzf kafka_2.11-1.0.0.tgz
```

Run the following from separate terminals:

```
cd kafka_2.11-1.0.0
bin/zookeeper-server-start.sh config/zookeeper.properties
```

```
cd kafka_2.11-1.0.0
bin/kafka-server-start.sh config/server.properties
```

In [1]:
!cd ~/software/kafka_2.11-1.0.0; \
    ./bin/kafka-topics.sh --zookeeper localhost:2181 --delete --topic test
!cd ~/software/kafka_2.11-1.0.0; \
    ./bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic test
!cd ~/software/kafka_2.11-1.0.0; \
    ./bin/kafka-topics.sh --list --zookeeper localhost:2181

Topic test is marked for deletion.
Note: This will have no impact if delete.topic.enable is not set to true.
Created topic "test".
__consumer_offsets
test


To delete a topic:

Add the following lines to `config/server.properties`: `delete.topic.enable=true`

In [ ]:
!cd ~/software/kafka_2.11-1.0.0 & bin/kafka-topics.sh --zookeeper localhost:2181 --delete --topic test

**Setup Confluent_Kafka**

First, install your own Anaconda to a local directory in your home on Palmetto. 

Next, perform the following steps

```
$ wget https://repo.continuum.io/archive/Anaconda3-5.0.1-Linux-x86_64.sh
$ sh Anaconda3-5.0.1-Linux-x86_64.sh
$ export PATH=/home/lngo/software/anaconda3/bin:$PATH
$ conda create --name kafka python=3.6
$ source activate kafka
$ conda install jupyter
$ python -m ipykernel install --prefix=/home/lngo/.local/ --name 'Python-Kafka-3.6'
$ conda install -c conda-forge python-confluent-kafka
$ conda install -c conda-forge tweepy
```

In [ ]:
from confluent_kafka import Producer
import sys

import logging
import json

from datetime import datetime

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

broker = 'localhost:9092'
topic = 'test'

# Producer configuration
# See https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md
conf = {'bootstrap.servers': broker}

# Create Producer instance
p = Producer(**conf)

# Optional per-message delivery callback (triggered by poll() or flush())
# when a message has been successfully delivered or permanently
# failed delivery (after retries).
def delivery_callback(err, msg):
    if err:
        sys.stderr.write('%% Message failed delivery: %s\n' % err)
    else:
        sys.stderr.write('%% Message key %s delivered to %s [%d]\n' % (msg.key(), msg.topic(), msg.partition()))

class StdOutListener(StreamListener):
    def on_data(self, data):
        try:
            jsonData = json.loads(data)
            p.produce(topic, json.dumps(data), key=str(jsonData['id']), callback=delivery_callback)
        except BufferError as e:
            sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))
        p.poll(0)
        return True
    def on_error(self, status):
        print (status)
        
# read cert (not on github)
keyFile = open('/home/lngo/.cert/Twitter','r')
CONSUMER_KEY = keyFile.readline().rstrip()
CONSUMER_SECRET = keyFile.readline().rstrip()
ACCESS_TOKEN_KEY = keyFile.readline().rstrip()
ACCESS_TOKEN_SECRET = keyFile.readline().rstrip()

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

while True:
    try:
        stream = Stream(auth, StdOutListener())
        stream.filter(track=['Clemson'])
    except IncompleteRead:
        print ("Lets try it again")
        continue
    except KeyboardInterrupt:
        stream.disconnect()
        break
        
# Wait until all messages have been delivered
sys.stderr.write('%% Waiting for %d deliveries\n' % len(p))
p.flush()

% Message key b'935205074590224384' delivered to test [0]
% Message key b'935205079136854016' delivered to test [0]
% Message key b'935205080797794305' delivered to test [0]
% Message key b'935205098933968896' delivered to test [0]
% Message key b'935205117766356993' delivered to test [0]
% Message key b'935205119498575873' delivered to test [0]
% Message key b'935205153896050688' delivered to test [0]
% Message key b'935205171415498754' delivered to test [0]
% Message key b'935205197533638657' delivered to test [0]
% Message key b'935205199123304449' delivered to test [0]
% Message key b'935205232283389952' delivered to test [0]
% Message key b'935205246539915264' delivered to test [0]
% Message key b'935205261689712641' delivered to test [0]
% Message key b'935205265716236288' delivered to test [0]
% Message key b'935205294899998720' delivered to test [0]
% Message key b'935205296452132865' delivered to test [0]
% Message key b'935205309475360771' delivered to test [0]
% Message key 

% Message key b'935206900018401280' delivered to test [0]
% Message key b'935206901675110403' delivered to test [0]
% Message key b'935206924009857025' delivered to test [0]
% Message key b'935206929311334401' delivered to test [0]
% Message key b'935206941814599681' delivered to test [0]
% Message key b'935206955114786816' delivered to test [0]
% Message key b'935206976509894659' delivered to test [0]
% Message key b'935206985976418305' delivered to test [0]
% Message key b'935206993261772800' delivered to test [0]
% Message key b'935207012488630272' delivered to test [0]
% Message key b'935207034152140800' delivered to test [0]
% Message key b'935207085767307264' delivered to test [0]
% Message key b'935207100556328960' delivered to test [0]
% Message key b'935207119455956992' delivered to test [0]
% Message key b'935207120777220096' delivered to test [0]
% Message key b'935207129153236992' delivered to test [0]
% Message key b'935207140138070017' delivered to test [0]
% Message key 

% Message key b'935208883898916864' delivered to test [0]
% Message key b'935208889842323456' delivered to test [0]
% Message key b'935208924873031680' delivered to test [0]
% Message key b'935208925946957826' delivered to test [0]
% Message key b'935208934150983682' delivered to test [0]
% Message key b'935208944439570439' delivered to test [0]
% Message key b'935208954078089217' delivered to test [0]
% Message key b'935208955231469568' delivered to test [0]
% Message key b'935208966509928448' delivered to test [0]
% Message key b'935208994687279105' delivered to test [0]
% Message key b'935209027855966208' delivered to test [0]
% Message key b'935209038459146240' delivered to test [0]
% Message key b'935209111729463296' delivered to test [0]
% Message key b'935209131836956674' delivered to test [0]
% Message key b'935209135557218304' delivered to test [0]
% Message key b'935209148739878912' delivered to test [0]
% Message key b'935209148995833862' delivered to test [0]
% Message key 